# Initialize environment

In [1]:
using Pkg
Pkg.activate("..")

using 
    Printf,
    Statistics,
    JLD2,
    FileIO, 
    Interpolations, 
    DataStructures,
    PyPlot,
    PyPlotPlus,
    SqueezeDispersion.Samoan

using LinearAlgebra: norm

usecmbright() # for plotting

┌ Info: Recompiling stale cache file /Users/gregorywagner/.julia/compiled/v1.0/JLD2/O1EyT.ji for JLD2 [033835bb-8acc-5ee8-8aae-3f567f8a3819]
└ @ Base loading.jl:1187
┌ Info: Recompiling stale cache file /Users/gregorywagner/.julia/compiled/v1.0/SqueezeDispersion/TsKDn.ji for SqueezeDispersion [b60af848-b6c1-11e8-3789-85de93651b40]
└ @ Base loading.jl:1187


# Load data

In [2]:
filepath = joinpath("..", "data", "samoanpassagedata.jld2")
ctd = load(filepath, "ctd")
vmp = load(filepath, "vmp")

ctdlon = ctd["lon"]
ctdlat = ctd["lat"]
ctdsigma4 = ctd["sigma4"]
ctddepth = ctd["depth"]

nz, nctd = size(ctdsigma4)
nvmp = length(vmp)
println("CTD data size: ndepth = $nz, nctd = $nctd, nvmp = $nvmp")

CTD data size: ndepth = 6001, nctd = 18, nvmp = 13


# Define density layers

In [3]:
sigmatop = 45.86
sigmabot = 45.96
dsigma = 0.005
layers = sigmatop:dsigma:sigmabot

layeredges = cat([sigmatop-dsigma/2], 0.5*(layers[2:end] + layers[1:end-1]), [sigmabot+dsigma/2], dims=1)
nlayers = length(layers)

println("nlayers = $nlayers")

nlayers = 21


# CTD thickness analysis

In [4]:
ctdthickness = zeros(nlayers, nctd)

for i = 1:nctd
    # Consider only non-NaN'd sigmas.
    goodidx = .!isnan.(ctdsigma4[:, i])
    goodsigma = ctdsigma4[goodidx, i]
    gooddepth = ctddepth[goodidx]
    nonsmoothsigma = deepcopy(goodsigma)
    smoothoverturns!(goodsigma)
    
    #println("(ictd $i) Rel norm of nonsmooth to smooth σ: $(norm(nonsmoothsigma-goodsigma) / norm(goodsigma))")
    
    interpsig = interpolate((goodsigma,), gooddepth, Gridded(Linear()))
    extrapsig = extrapolate(interpsig, NaN)
    edgedepths = extrapsig(layeredges)
    ctdthickness[:, i] = edgedepths[2:end] - edgedepths[1:end-1]   
end

# VMP thickness analysis
layersamples, layerkappa, layerepsilon, vmpthickness, layerinvN2 = vmplayeranalysis(vmp, layeredges)

# Mean thickness
meanthickness = samoanavg(cat(vmpthickness, ctdthickness, dims=2))
meanvmpthickness = samoanavg(vmpthickness)
meanctdthickness = samoanavg(ctdthickness)
meanepsilon = samoanavg(layerepsilon)
meaninvN2 = samoanavg(layerinvN2)

mixingcoeff = 0.2
kappa_squeeze0 = effectivekappa(mixingcoeff, layerepsilon, vmpthickness; avgh=meaninvN2)
kappa_squeeze1 = effectivekappa(layerkappa, vmpthickness; avgh=meanthickness)
kappa_naive = samoanavg(layerkappa)

gravity = 9.81
scaled_flux0 = @. gravity*mixingcoeff*layerepsilon*meanvmpthickness
scaled_flux1 = @. layerkappa*meanvmpthickness/vmpthickness

for (i, n) in enumerate(sum(layersamples, dims=2))
    @printf("sigma: %.3f, mean thickness: %.2f m, mean ε: %.2e, samples: %d\n", 
            layers[i], meanvmpthickness[i], avgepsilon[i], n)
end

println("\nMedian layer thickness: $(median(meanvmpthickness))")

BoundsError: BoundsError: attempt to access ([69.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 58.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 50.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 87.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 108.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 79.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 49.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 133.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 117.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 103.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 120.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 40.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 27.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 12.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 12.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 60.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 22.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 32.0 0.0 0.0 0.0 0.0 0.0 0.0 0.

In [5]:
# Plot

## Parameters
      ms0 = 4
      ms1 = 6
      ms2 = 10
    alpha = 0.6
  legsize = 10
xlabelpad = 12.0
    maxms = 10
 vmpcolor = "xkcd:maroon"
 ctdcolor = "xkcd:dark cyan"
vmpsymbol = "^"
ctdsymbol = "+"
xtextpos = 0.03
ytextpos = 0.93
fs = 16

fig, axs = subplots(ncols=3, figsize=(10, 5), sharey=true,  gridspec_kw=Dict("width_ratios"=>[1, 2, 1]))

# Thickness
sca(axs[1])
plot(vmpthickness, layers, vmpsymbol, color=vmpcolor, markeredgecolor="None", alpha=alpha, ms=ms0)
plot(ctdthickness, layers, ctdsymbol, color=ctdcolor, markeredgecolor=ctdcolor, alpha=alpha, ms=ms0)
plot(meanthickness, layers, "s", color="xkcd:soft green", markeredgecolor="None", alpha=0.8, ms=ms2, label="Avg")

# For labeling
plot(vmpthickness[1], layers[1], vmpsymbol, color=vmpcolor, markeredgecolor="None", alpha=alpha, ms=ms0, label="VMP")
plot(ctdthickness[1], layers[1], ctdsymbol, color=ctdcolor, markeredgecolor=ctdcolor, alpha=alpha, ms=ms1, label="CTD")

xscale("log")
xlim(1, 400)
cornerspines()

xticks([1, 10, 100])
xlabel("Thickness \$ h \$ (m)", labelpad=12.0)
ylabel("Layer potential density \$ \\rho \$ (\$ \\mathrm{kg/m^3} \$)", labelpad=xlabelpad)
legend(loc="upper left", bbox_to_anchor=(0.04, 0.92), 
       prop=Dict("size"=>legsize), markerscale=1.0, markerfirst=true, frameon=true,
       handletextpad=0.05)


# Kappa
sca(axs[2])
scaledms = min.(maxms, @. ms0 * (meanthickness / vmpthickness)^(1/2))
        
plot(layerkappa, layers, "o", color="xkcd:bright blue", markeredgecolor="None", alpha=alpha, ms=ms0)
plot(scaled_flux0, layers, "*", color="xkcd:rose", markeredgecolor="None", alpha=alpha, ms=ms1)

plot(kappa_naive, layers, "s", color="xkcd:rust", markeredgecolor="None", alpha=0.6, ms=ms2, label="Naive avg")
plot(kappa_squeeze0, layers, "s", color="xkcd:black", markeredgecolor="None", alpha=0.7, ms=ms2, label="Correct flux calc")
plot(kappa_squeeze1, layers, "s", color="xkcd:bright blue", markeredgecolor="None", alpha=0.3, ms=ms2, label="Squeeze avg")

plot(layerkappa[1], layers[1], "o", color="xkcd:bright blue", markeredgecolor="None", alpha=0.4, ms=ms0, 
     label=L"\kappa_i(\rho)")
plot(scaled_flux0[1], layers[1], "*", color="xkcd:rose", markeredgecolor="None", alpha=alpha, ms=ms1,
        label=L"\Gamma \epsilon(\rho) \langle h \rangle") 
        #label=L"\kappa_i(\rho) \langle h \rangle / h_i(\rho)")

xscale("log")
xlim(1e-11, 1e-1)
xticks([1e-6, 1e-5, 1e-4, 1e-3, 1e-2])
bottomspine()
axs[2][:tick_params](axis="y", which="both", labelleft=false, left=false)
xlabel("Layer-averaged \$ \\kappa \$ (\$ \\mathrm{m^2/s} \$)", labelpad=(xlabelpad-1.5))
legend(loc="upper right", bbox_to_anchor=(1.0, 0.98), 
       prop=Dict("size"=>legsize), markerfirst=false, markerscale=1.0, borderaxespad=0.0, frameon=true, handletextpad=0.05)

# Enhancement
sca(axs[3])
plot(kappa_squeeze0./kappa_naive, layers, "s", color="xkcd:purplish blue", markeredgecolor="None", ms=ms2, alpha=0.7, label="2D approx")
plot([1, 1], layeredges[[1, end]], "k-", linewidth=0.5, alpha=0.5)

cornerspines(side="right")
axisright()
xscale("log")
xlim(0.4, 3.3)
xlabel("\$ \\kappa_e / \\langle \\kappa \\rangle \$", labelpad=(xlabelpad+.5))
ylabel("Layer potential density \$ \\rho \$ (\$ \\mathrm{kg/m^3} \$)", labelpad=12.0)

xticks([0.5, 1, 2, 3], [L"\tfrac{1}{2}", "1", "2", "3"])
scalartickformat(:x)

ylim(45.855, 45.963)
invertaxis()

text(xtextpos, ytextpos, "(c)", transform=axs[1][:transAxes], ha="left", va="bottom", fontsize=fs)
text(xtextpos, ytextpos, "(d)", transform=axs[2][:transAxes], ha="left", va="bottom", fontsize=fs)
text(xtextpos, ytextpos, "(e)", transform=axs[3][:transAxes], ha="left", va="bottom", fontsize=fs)

tight_layout(w_pad=2.0)

plotname = "samoanresults.png"
savefig(plotname, dpi=960)

UndefVarError: UndefVarError: meanthickness not defined

In [6]:
#cp(plotname, joinpath("..", "draft", plotname), remove_destination=true)